In [ ]:
# install pyscf
# will simplify setup later...
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.plot3dtree import plot3dtree

# basis
mol_h2o = gto.M(
    verbose = 0,
    atom = '''
    o    0    0.       0.
    h    0    -0.757   0.587
    h    0    0.757    0.587''',
    basis = 'ccpvdz') 

dom = numpy.array([-5, 5, -5, 5, -5, 5])
tol = 1.0e-4
p = 8
checkpts = numpy.array([[ 0,    0.,       0.],
                        [ 0,    -0.757,   0.587],
                        [ 0,    0.757,    0.587]])
ifcoeffs = True

# tree1
def pyscffunc1(x, y, z, mol):
  return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
# nd1 = 24 # ccpvdz
nd1 = 58 # ccpvtz
func1 = lambda x, y, z: pyscffunc1(x, y, z, mol_h2o)
tree1_h2o = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                      
  }
tree1_h2o, rint = buildBreadthFirst3d(tree1_h2o, func1)
numpts = 51 # this needs to be consistent with the resolution in plot3dtree.m
xx, yy, zz = numpy.meshgrid(numpy.linspace(dom[0],dom[1],numpts),numpy.linspace(dom[2],dom[3],numpts),numpy.linspace(dom[4],dom[5],numpts),indexing='ij')
v1 = func1(xx.flatten(),yy.flatten(),zz.flatten())
savemat('tree1.mat', {'numpts': numpts, 'v': v1, 'xx': xx, 'yy': yy, 'zz': zz, 'rint': rint, 'fdomain': tree1_h2o['domain'], 'fn': tree1_h2o['n'], 'flevel': tree1_h2o['level'], 'fchildren': tree1_h2o['children'], 'fheight': tree1_h2o['height'], 'fid': tree1_h2o['id'], 'frint': tree1_h2o['rint'], 'ftol': tree1_h2o['tol'], 'fcheckpts': tree1_h2o['checkpts']})

# tree2
nd2 = nd1**2
def pyscffunc2(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows, ncols**2))
  for j in range(ncols):
    for k in range(ncols):
        vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
  return vals
func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_h2o)
tree2_h2o = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                  
  }
numpts = 51 # this needs to be consistent with the resolution in plot3dtree.m
tree2_h2o, rint = buildBreadthFirst3d(tree2_h2o, func2)
xx, yy, zz = numpy.meshgrid(numpy.linspace(dom[0],dom[1],numpts),numpy.linspace(dom[2],dom[3],numpts),numpy.linspace(dom[4],dom[5],numpts),indexing='ij')
v2 = func2(xx.flatten(),yy.flatten(),zz.flatten())
savemat('tree2.mat', {'numpts': numpts, 'v': v2, 'xx': xx, 'yy': yy, 'zz': zz, 'rint': rint, 'fdomain': tree2_h2o['domain'], 'fn': tree2_h2o['n'], 'flevel': tree2_h2o['level'], 'fchildren': tree2_h2o['children'], 'fheight': tree2_h2o['height'], 'fid': tree2_h2o['id'], 'frint': tree2_h2o['rint'], 'ftol': tree2_h2o['tol'], 'fcheckpts': tree2_h2o['checkpts']})


# tree3
nd3 = 1
def pyscffunc3(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows))
  for j in range(ncols):
    vals = vals + valstmp[:, j]**2
  return vals
func3 = lambda x, y, z: pyscffunc3(x, y, z, mol_h2o)
tree3_h2o = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                     
  }
numpts = 51 # this needs to be consistent with the resolution in plot3dtree.m
tree3_h2o, rint = buildBreadthFirst3d(tree3_h2o, func3)
xx, yy, zz = numpy.meshgrid(numpy.linspace(dom[0],dom[1],numpts),numpy.linspace(dom[2],dom[3],numpts),numpy.linspace(dom[4],dom[5],numpts),indexing='ij')
v3 = func3(xx.flatten(),yy.flatten(),zz.flatten())
savemat('tree3.mat', {'numpts': numpts, 'v': v3, 'xx': xx, 'yy': yy, 'zz': zz, 'rint': rint, 'fdomain': tree3_h2o['domain'], 'fn': tree3_h2o['n'], 'flevel': tree3_h2o['level'], 'fchildren': tree3_h2o['children'], 'fheight': tree3_h2o['height'], 'fid': tree3_h2o['id'], 'frint': tree3_h2o['rint'], 'ftol': tree3_h2o['tol'], 'fcheckpts': tree3_h2o['checkpts']})


plot3dtree(tree2_h2o)

In [ ]:
# basis
mol_c2h4 = gto.Mole()
mol_c2h4.atom = [
        ["C", (-0.65830719,  0.61123287, -0.00800148)],
        ["C", ( 0.73685281,  0.61123287, -0.00800148)],
        ["H", ( 1.43439081,  1.81898387, -0.00800148)],
        ["H", (-1.35568919,  1.81920887, -0.00868348)],
        ["H", (-1.20806619, -0.34108413, -0.00755148)],
        ["H", ( 1.28636081, -0.34128013, -0.00668648)],]
mol_c2h4.basis = {'H': 'cc-pvdz',
                  'C': 'cc-pvdz',}
mol_c2h4.build()

dom = numpy.array([-5, 5, -5, 5, -5, 5])
tol = 1.0e-4
p = 8
checkpts = numpy.array([[ -0.65830719,  0.61123287, -0.00800148],
                        [  0.73685281,  0.61123287, -0.00800148],
                        [  1.43439081,  1.81898387, -0.00800148],
                        [ -1.35568919,  1.81920887, -0.00868348],
                        [ -1.20806619, -0.34108413, -0.00755148],
                        [  1.28636081, -0.34128013, -0.00668648]])

# tree4
nd = 48
# def pyscffunc4(x, y, z, mol):
#   return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))

nd = nd**2
def pyscffunc_c2h4(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows, ncols**2))
  for j in range(ncols):
    for k in range(ncols):
        vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
  return vals

func_c2h4 = lambda x, y, z: pyscffunc_c2h4(x, y, z, mol_c2h4)
tree_c2h4 = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                      
  }
tree_c2h4, rint = buildBreadthFirst3d(tree_c2h4, func_c2h4)

plot3dtree(tree_c2h4)

In [ ]:
# basis
mol_ch4 = gto.Mole()
mol_ch4.atom = [ # CH4
    ['H',    0.0000,    0.9870,    0.0000],
    ['C',    1.0390,    0.9870,    0.0000],
    ['H',    1.0390,    1.9740,    0.0000],
    ['H',    1.0390,    0.0000,    0.0000],
    ['H',    2.0260,    0.9870,    0.0000]]
mol_ch4.basis = {'H': 'cc-pvdz',
             'C': 'cc-pvdz',}
mol_ch4.build()

dom = numpy.array([-5, 5, -5, 5, -5, 5])
tol = 1.0e-4
p = 8
checkpts = numpy.array([[ 0.0000,    0.9870,    0.0000],
                        [ 1.0390,    0.9870,    0.0000],
                        [ 1.0390,    1.9740,    0.0000],
                        [ 1.0390,    0.0000,    0.0000],
                        [ 2.0260,    0.9870,    0.0000]])

# tree5
nd = 34 # for cc-pvdz
# nd5 = 86 # for cc-pvtz
# def pyscffunc5(x, y, z, mol):
#   return numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))

nd = nd**2
def pyscffunc_ch4(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows, ncols**2))
  for j in range(ncols):
    for k in range(ncols):
        vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
  return vals

func_ch4 = lambda x, y, z: pyscffunc_ch4(x, y, z, mol_ch4)
tree_ch4 = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                           
  }
tree_ch4, rint = buildBreadthFirst3d(tree_ch4, func_ch4)

plot3dtree(tree_ch4)
